In [ ]:
import torch
from datasets import load_dataset
from transformers import (
    RobertaTokenizerFast,
    RobertaForSequenceClassification,
    TrainingArguments,
    Trainer,
    AutoConfig,
)
import pandas as pd
from datasets import Dataset

In [ ]:
model_id = "roberta-base"
DATA_PATH = "..//data//refined_dataset.csv"

In [ ]:
df = pd.read_csv(DATA_PATH, delimiter="|")
new_dataset = df[['label', 'body']].copy()
dataset = Dataset.from_pandas(new_dataset)
dataset.num_classes = 3
dataset

In [ ]:
train_val_data = dataset.train_test_split(test_size=64, seed=42, shuffle=True)
train_dataset = train_val_data["train"].shuffle()
test_val_data = train_val_data["test"].train_test_split(test_size=32, seed=42, shuffle=True)
val_dataset = test_val_data["train"].shuffle()
test_dataset = test_val_data["test"].shuffle()
print(train_dataset.num_rows)
print(val_dataset.num_rows)
print(test_dataset.num_rows)

In [ ]:
tokenizer = RobertaTokenizerFast.from_pretrained(model_id)

def tokenize(batch):
    return tokenizer(batch["body"], padding=True, truncation=True, max_length=256)

train_dataset = train_dataset.map(tokenize, batched=True, batch_size=len(train_dataset))
val_dataset = val_dataset.map(tokenize, batched=True, batch_size=len(val_dataset))
test_dataset = test_dataset.map(tokenize, batched=True, batch_size=len(test_dataset))

In [ ]:
train_dataset.set_format("torch", columns=["input_ids", "attention_mask", "label"])
val_dataset.set_format("torch", columns=["input_ids", "attention_mask", "label"])
test_dataset.set_format("torch", columns=["input_ids", "attention_mask", "label"])

In [ ]:
train_dataset

In [ ]:
id2label = {0:"student", 1:"corporate", 2:"researcher"}
config = AutoConfig.from_pretrained(model_id)
config.update({"id2label": id2label})

In [ ]:
ADAPTER_PATH = "./adapters"
OUTPUT_PATH = "./output"
LOGS_PATH = "./logs"

In [ ]:
model = RobertaForSequenceClassification.from_pretrained(model_id, config=config)

training_args = TrainingArguments(
    num_train_epochs=5,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    gradient_accumulation_steps=1,
    eval_strategy="epoch",
    logging_strategy="steps",
    logging_steps=10,
    learning_rate=5e-5,
    weight_decay=0.01,
    warmup_steps=500,
    optim="adamw_torch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    save_total_limit=2,
    report_to=[],
    output_dir=ADAPTER_PATH,
    logging_dir=LOGS_PATH,
    push_to_hub=False,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)

In [ ]:
trainer.train()